In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
import os
from dotenv import load_dotenv

import pymysql
# 상위 폴더(../)에 있는 `.env` 파일 로드
dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))
load_dotenv(dotenv_path)
## mysql 연결
os.getenv('user_id')
os.getenv('user_password')

mysql_url = f"jdbc:mysql://{os.getenv('host_ip')}:3306/{os.getenv('DATABASE')}?useSSL=false&allowPublicKeyRetrieval=true&useUnicode=true&characterEncoding=UTF-8"

# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS to MySQL") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

# HDFS에 있는 모든 Parquet 파일 로드
df = spark.read.parquet("hdfs:///data/")
print(df.count())

## 시간 형식 수정(yyyy.mm.dd HH:MM:SS)
convert_cols = ['comment_publish_date', 'publish_date']
for c in convert_cols:
    df = df.withColumn(c
                     , to_timestamp(col('comment_publish_date'), "yyyy-MM-dd'T'HH:mm:ss'Z'")
                     )
## 결측치 제거
df = df.dropna()

# PyMySQL로 MySQL 연결 (SQL 실행을 위해)
conn = pymysql.connect(
    host=os.getenv('host_ip'),
    user=os.getenv('user_id'),
    password=os.getenv('user_password'),
    database=os.getenv('DATABASE'),
)

cursor = conn.cursor()

# ✅ MySQL 테이블 `TRUNCATE`
truncate_query = "TRUNCATE TABLE tbCrawled_Youtube"
cursor.execute(truncate_query)

conn.commit()
cursor.close()
conn.close()


os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars mysql-connector-java-5.1.49.jar pyspark-shell'
df.write.format('jdbc')\
    .options(
        url = mysql_url
        , driver = 'com.mysql.jdbc.Driver'
        , dbtable = 'tbCrawled_Youtube'
        , user = os.getenv('user_id')
        , password = os.getenv('user_password')
    )\
    .mode('append')\
    .save()

25/02/04 13:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


136568


Py4JJavaError: An error occurred while calling o52.save.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:102)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:102)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:217)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:221)
	at org.apache.spark.sql.execution.da

In [2]:
spark.stop()


In [51]:
import pandas as pd
import glob

# 파일 경로 설정
df_list = [pd.read_parquet(i) for i in glob.glob('../../airflow/danawa_data/*.parquet')]
df_list
pd.concat(df_list)

,scoring,market,purchasing_date,review_title,review_content,item
0,100점,G마켓,2024.07.19.,\n잘 받았습니다.\n,뜯어서 확인해보니 진짜 이쁘네요ㅎㅎ 생각외로 무게도 엄청 가볍고 특히 그레이 색상이...,"삼성전자 갤럭시Z 폴드6 256GB, 자급제 (자급제 공기계)"
1,100점,롯데하이마트,2024.12.26.,\n드디어 폴드6를 구입하였습니다.\n,드디어 폴드6를 구입하였습니다.할인 받아서 좋은 가격에 구입할 수 있어서 매우 만족...,"삼성전자 갤럭시Z 폴드6 256GB, 자급제 (자급제 공기계)"
2,100점,롯데하이마트,2024.09.13.,\n2021년 처음 맞이한 Z폴드3를 3년 사\n,2021년 처음 맞이한 Z폴드3를 3년 사용하였습니다.바꿔보상이 올해가 Z폴드3를 ...,"삼성전자 갤럭시Z 폴드6 256GB, 자급제 (자급제 공기계)"
3,80점,G마켓,2024.07.22.,\n갤럭시 폴더6 네이비 추천합니다.\n,갤럭시폴더6 추천합니다. 이번 사전예약으로 이전 z폴더에 이어 두번째 폴더폰인데 메...,"삼성전자 갤럭시Z 폴드6 256GB, 자급제 (자급제 공기계)"
4,100점,G마켓,2024.11.10.,\n너무좋아요.\n,사과농장 짓고 십여년 지내다가 이번에 좋은 기회로 기기변경했어요. 기기 동작이 빠릿...,"삼성전자 갤럭시Z 폴드6 256GB, 자급제 (자급제 공기계)"
...,...,...,...,...,...,...
1076,100점,11번가,2024.07.30.,\n최고예요\n,역시는 역시입니다z플립3에서 z플립6로 갈아탔습니다3보다 그립김도 좋고힌지부분이 보...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1077,100점,11번가,2024.07.31.,\n최고예요\n,아빠가 오랜 기간동안 플립2를 사용하셨어요. 플립2가 만족스러워서 폴드를 샀다가도 ...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1078,100점,11번가,2024.08.01.,\n최고예요\n,거의 3년만에 바꾸는 폰인데 일단 이쁘고 성능 좋아서 샀어요!케이스는 저렴이 샀는데...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1079,100점,11번가,2024.08.02.,\n최고예요\n,"너무 이쁘고 가볍고 카메라 화질도 좋아요! 플립3에서 4,5 안갈아타고 6으로 하길...","삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"


In [45]:
import pandas as pd
import os
import glob
print(os.getcwd())

#pd.read_parquet('../../airflow/danawa_data/*.parquet')
pd.read_parquet('./danawa_review_플립6+0.parquet')


/home/lab13/project/YJ


,scoring,market,purchasing_date,review_title,review_content,item
0,80점,삼성닷컴,2024.08.17.,\n곧 나올 아이폰 16을 살지 z플립6를 살지 고민하다가 플립으로 ...\n,곧 나올 아이폰 16을 살지 z플립6를 살지 고민하다가 플립으로 골랐습니다.\n\n...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1,100점,삼성닷컴,2024.08.27.,\n중동점 매장에서 받아왔는데 아주 예쁘고 건실한 핸드폰을 받아 ...\n,중동점 매장에서 받아왔는데 아주 예쁘고 건실한 핸드폰을 받아 좋았습니다\n수령만 하...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
2,100점,삼성닷컴,2024.08.13.,\n3년 전 Z플립3에 반해서 홀린듯이 구입하고 3년 동안 잘 썼어요...\n,3년 전 Z플립3에 반해서 홀린듯이 구입하고 3년 동안 잘 썼어요ㅎㅎㅎ 기능도 기능...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
3,100점,삼성닷컴,2024.08.28.,\n#삼성전자이벤트참여z플립3 쓰다가 이번에 바꿔보상 추가금액 이벤트...\n,#삼성전자이벤트참여\n\nz플립3 쓰다가 이번에 바꿔보상 추가금액 이벤트 해서 핸드...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
4,100점,G마켓,2024.07.29.,"\n이번에 어머니 전화기를 갤럭시 Z플립 6로 바꾸어 드렸는데, 매우 만족하시네요.\n","어머니께서 저와 와이프, 딸네미가 차고 있는 걸럭시 워치에 대해서 관심을 보이시길래...","삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
...,...,...,...,...,...,...
1076,100점,11번가,2024.07.30.,\n최고예요\n,역시는 역시입니다z플립3에서 z플립6로 갈아탔습니다3보다 그립김도 좋고힌지부분이 보...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1077,100점,11번가,2024.07.31.,\n최고예요\n,아빠가 오랜 기간동안 플립2를 사용하셨어요. 플립2가 만족스러워서 폴드를 샀다가도 ...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1078,100점,11번가,2024.08.01.,\n최고예요\n,거의 3년만에 바꾸는 폰인데 일단 이쁘고 성능 좋아서 샀어요!케이스는 저렴이 샀는데...,"삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"
1079,100점,11번가,2024.08.02.,\n최고예요\n,"너무 이쁘고 가볍고 카메라 화질도 좋아요! 플립3에서 4,5 안갈아타고 6으로 하길...","삼성전자 갤럭시Z 플립6 256GB, 자급제 (자급제 공기계)"


In [47]:
import pandas as pd
import os
import glob
print(os.getcwd())

#pd.read_parquet('../../airflow/danawa_data/*.parquet')
pd.read_parquet('./danawa_review_플립6+1.parquet')


/home/lab13/project/YJ


,scoring,market,purchasing_date,review_title,review_content,item
0,100점,삼성닷컴,2024.08.15.,\n저는 공홈에거 보상판매 직업 픽업방식으로 구매했어요. 저렴하게 구매하...\n,저는 공홈에거 보상판매 직업 픽업방식으로 구매했어요. 저렴하게 구매하고 기다리비 않...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
1,20점,G마켓,2024.11.27.,\n폰은 예쁜데 서비스가 아쉽네요\n,지마켓 라이브로 합리적인 가격에 구매했습니다~ 배송도 이틀인가만에 와서 좋았는데 개...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
2,100점,G마켓,2024.07.23.,\n배송 빠르네요. 주문하고 다음날 받았습니다.\n,Z플립5와 외관은 거의 차이가 없어 보이지만 속이 많이 바뀌어서 구입하게 되었네요....,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
3,100점,G마켓,2024.11.12.,\n5년 만의 새 핸드폰\n,오랜만에 핸드폰 바꾸고 설레는 마음으로 이것저것 만져보는데 참 세월이 빠르게 가고 ...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
4,100점,삼성닷컴,2024.08.13.,\n디자인도 이쁘고 색상도 이쁩니다.초4큰아이 생일 선물로 구매했...\n,디자인도 이쁘고 색상도 이쁩니다.\n초4큰아이 생일 선물로 구매했습니다.\n젊은 사...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
...,...,...,...,...,...,...
576,100점,11번가,2024.07.26.,\n최고예요\n,z플립3에서 z플립 6로 변경하는 유저입니다. 확실히 z플립 3에 비해 6는 힌지가...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
577,80점,옥션,2025.01.28.,\n잘 받았습니다 좋아요\n,잘 받았습니다 좋아요,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
578,100점,11번가,2024.07.27.,\n최고예요\n,갤럭시 플립 3 사용중에 액정 수리비가 너무 많이 나와서 새로나온 플립6 구입했는데...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"
579,80점,11번가,2024.07.30.,\n좋아요\n,바꿔보상 받아서 체감상 조금은 저렴한 느낌이지만 그외의 혜택은 없어서 조금 아쉬웠어...,"삼성전자 갤럭시Z 플립6 512GB, 자급제 (자급제 공기계)"


In [49]:
import pandas as pd
import os
import glob
print(os.getcwd())

#pd.read_parquet('../../airflow/danawa_data/*.parquet')
pd.read_parquet('./danawa_review_폴드6+2.parquet')


/home/lab13/project/YJ


,scoring,market,purchasing_date,review_title,review_content,item
0,100점,삼성닷컴,2024.07.31.,\n갤럭시 z폴드3을 사용하다가 갤럭시 z폴드6으로 갈아 타니 신세계 그...\n,갤럭시 z폴드3을 사용하다가 갤럭시 z폴드6으로 갈아 타니 신세계 그 자체 였어요....,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
1,100점,G마켓,2025.02.03.,\n1월25일 토요일 수령\n,"플립과 고민하다 심사숙고 해서 폴드6 선택!!설연휴라 좀 쉬고, 2월 2일 에서야 ...","삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
2,100점,삼성닷컴,2024.07.28.,\n폴드4에이어 두번째 폴더블 폰인데 너무 만족합니다. 일단 너무 가...\n,폴드4에이어 두번째 폴더블 폰인데 너무 만족합니다. 일단 너무 가벼워진것과 디스플...,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
3,100점,삼성닷컴,2024.07.29.,\n폴드6 사용 3일차입니다.22 울트라 256기가를 쓰고 있었는데 저...\n,폴드6 사용 3일차입니다.\n22 울트라 256기가를 쓰고 있었는데 저장공간이 점점...,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
4,100점,G마켓,2024.11.07.,\n아주 만족합니다\n,폴드se를 사려고 하다가 1테라도 없고 가격도 비싸고 해서 그냥 6로 왔는데 만족합...,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
...,...,...,...,...,...,...
107,100점,11번가,2024.07.18.,\n최고예요\n,노트9에서 6년만에 기변이네요..아직도 짱짱하지만폴드 한번 써볼려고 사전구매진행! ...,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
108,100점,11번가,2024.07.18.,\n최고예요\n,폴드4쓰다가 5거르고 6샀는데 성능도 좋고 가운데 선이 잘안보여서 좋음 이제 7이나...,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
109,60점,11번가,2024.07.18.,\n보통이에요\n,그전보다 가벼고 슬림해서 좋아요,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
110,100점,11번가,2024.07.23.,\n최고예요\n,이쁘고 화면 넓고 좋네요.무게도 가벼워서 더 좋아요.,"삼성전자 갤럭시Z 폴드6 1TB, 자급제 (자급제 공기계)"
